In [1]:

# load model

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..','fly_model'))
sys.path.insert(0, parent_dir)
import numpy as np
from Skin import Skin
import plotly.graph_objects as go
from Joint import Joint
from Frame import Frame
from Camera import Camera
import scipy

import numpy as np
import Plotter
from Skeleton import Skeleton
from Skin import Skin
import plotly.graph_objects as go
import Utils
import pandas as pd
%matplotlib qt

path_to_mesh = 'G:/My Drive/Research/gaussian_splatting/mesh'
skin_parts = ['body','right_wing','left_wing']
scale = 1/1000
skin_translation = np.array([-0.1-1,0,1])*scale

# initilize skeletone, joints and bones
# body angles - yaw,pitch,roll
# wing angles - phi, psi, theta
pitch_body = 0


root_no_bone = Joint([1,0,0],[0,-pitch_body,0],parent = None, end_joint_of_bone = False, scale = scale)
neck = Joint([0.6,0,0.3],[0,pitch_body,0],parent = root_no_bone, end_joint_of_bone = False, scale = scale)
neck_thorax =  Joint([0.6,0,0.3],[0,-25,0], parent = root_no_bone, end_joint_of_bone = False, scale = scale)
head  =Joint([0.3,0,0],[0,0,0], parent = neck, scale = scale)
thorax  =Joint([-1,0,0],[0,25,0], parent= neck_thorax ,scale = scale)
abdomen = Joint([-1.3,0,0],[0,0,0], parent = thorax, scale = scale)
right_sp_no_bone = Joint([0,0,0.3],[0,pitch_body,0],parent = root_no_bone, end_joint_of_bone = False , scale = scale, color = 'red', rotation_order = 'zxy')
right_wing_root = Joint([0,-0.3,0],[0,0,0], parent = right_sp_no_bone, end_joint_of_bone = False, scale = scale, color = 'red',rotation_order = 'zxy')
right_wing_tip = Joint([0,-2.2,0],[0,0,0], parent = right_wing_root, scale = scale, color = 'red',rotation_order = 'zxy')
left_sp_no_bone = Joint([0,0,0.3],[0,pitch_body,0], parent = root_no_bone, end_joint_of_bone = False, scale = scale, color = 'blue',rotation_order = 'zxy')
left_wing_root = Joint([0,0.3,0],[0,0,0],parent = left_sp_no_bone, end_joint_of_bone = False, scale = scale, color = 'blue',rotation_order = 'zxy')
left_wing_tip = Joint([0,2.2,0],[0,0,0], parent =left_wing_root, scale = scale, color = 'blue',rotation_order = 'zxy')


body = Skin(f'{path_to_mesh}/body.stl',scale = 1.1,color = 'lime')
right_wing = Skin(f'{path_to_mesh}/right_wing.stl',scale = 1.1,constant_weight = right_wing_root,color = 'crimson')
left_wing = Skin(f'{path_to_mesh}/left_wing.stl',scale = 1.1, constant_weight = left_wing_root,color = 'dodgerblue')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


d:\Documents\gaussian_splat\model\fly_model\Skin.py:21: RuntimeWarning:

invalid value encountered in divide



In [2]:
# initilize joints
joints_of_bone = root_no_bone.get_and_assign_bones()
[skin.add_bones(joints_of_bone) for skin in  [body, right_wing,left_wing]]
[skin.translate_ptcloud_skin(skin_translation) for skin in  [body, right_wing,left_wing]]
body.calculate_weights_dist(body.bones[0:3])
right_wing.calculate_weights_constant()
left_wing.calculate_weights_constant()

joint_list = root_no_bone.get_list_of_joints()
list_joints_pitch_update = [neck,right_sp_no_bone,left_sp_no_bone]


all_skin_points = [skin.rotate_skin_points() for skin in [body,right_wing,left_wing]]
all_skin_normals = [skin.rotate_skin_normals() for skin in [body,right_wing,left_wing]]




In [3]:
# load frames and cameras

path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'
frames = list(range(900,910,1))
image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]

frames = {f'{im_name}.jpg':Frame(path,im_name,idx) for idx,im_name in enumerate(image_name)}
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']


In [4]:
# hull reconstruction 

def get_hull_part(part):
    points_3d_frame = points_3d[part][points_3d[part]['frame'] == 900]
    real_coord_frame = real_coord[real_coord[:,3] == 900,:]
    points_in_ew_frame = np.array([real_coord_frame[points_3d_frame[ax] - 1,idx] for idx,ax in enumerate(['X','Y','Z'])]).T
    points_in_ew_frame  = np.column_stack((points_in_ew_frame,np.arange(1,points_in_ew_frame.shape[0] + 1)))
    return points_in_ew_frame
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}

hull = [get_hull_part(part) for part in list(points_3d.keys())]

In [5]:
frame_number = 900 
frame_names = ['P900CAM1.jpg','P900CAM2.jpg','P900CAM3.jpg','P900CAM4.jpg']
frame_names = [f'P{frame_number}CAM{idx}.jpg' for idx in range(1,5)]

camera_pixel = np.vstack([frames[frame].camera_center_to_pixel_ray(frames[frame].cm) for frame in  frame_names])
camera_center = np.vstack([frames[frame].X0.T for frame in  frame_names])
rot_mat_ew_to_lab = frames['P900CAM1.jpg'].rotation_matrix_from_vectors(frames['P900CAM1.jpg'].R[2,:], [0,0,1])
cm_point = Utils.triangulate_least_square(camera_center,camera_pixel)
cm_point_lab = np.squeeze(np.dot(rot_mat_ew_to_lab,cm_point[:,np.newaxis]).T)




In [8]:

fig = go.Figure()
Plotter.plot_skeleton(body.bones,fig,marker_dict = {'size': 3},line_dict ={'width': 3})
Plotter.plot_skin(fig,all_skin_points[0],body,'body',color = body.weights[::10,0])
Plotter.plot_skin(fig,all_skin_points[1],right_wing,'right wing')
Plotter.plot_skin(fig,all_skin_points[2],left_wing,'left wing')

fig.show()

In [7]:
# update model position

pitch = -25
root_no_bone.set_local_translation(cm_point_lab + np.array([-0.2,-0.4,0.22])*1/1000)
root_no_bone.set_local_rotation([249,pitch,0])
[joint.set_local_rotation([0,-pitch,0]) for joint in list_joints_pitch_update]
thorax.set_local_rotation([0,-10,0])
right_wing_root.set_local_rotation([-27,-120,10])
left_wing_root.set_local_rotation([27,-120,-15])
[joint.update_rotation() for joint in joint_list]

all_skin_points = [skin.rotate_skin_points() for skin in [body,right_wing,left_wing]]
all_skin_normals = [skin.rotate_skin_normals() for skin in [body,right_wing,left_wing]]



In [9]:
# rotate to easy wand and project to 2d
import matplotlib.pyplot as plt


frame_name = 'P900CAM2.jpg'
all_skin = np.vstack(all_skin_points)
all_skin_in_ew = np.dot(rot_mat_ew_to_lab.T,all_skin.T).T
all_skin_in_ew_homo = frames[frame_name].homogenize_coordinate(all_skin_in_ew)
projeced = frames[frame_name].project_on_image(all_skin_in_ew_homo)

plt.imshow(frames[frame_name].image_no_bg)
plt.scatter(projeced[:,0],projeced[:,1],color = 'black')


In [17]:
cm_point_lab[np.newaxis]

array([[-0.00134725,  0.00580915,  0.00811845]])

In [18]:
# rotate camera to lab
camera_center_lab = np.dot(rot_mat_ew_to_lab,camera_center.T).T
camera_pixel_lab = np.dot(rot_mat_ew_to_lab,camera_pixel.T).T


vector_dir = np.array(camera_pixel_lab - camera_center_lab )[:,0:3]
vector_dir_norm= (vector_dir.T/np.linalg.norm(vector_dir,axis = 1)).T


camera_pixel_lab  = camera_center_lab + vector_dir_norm*np.array([[0.3,0.22,0.22,0.22]]).T
camera_center_lab  = camera_center_lab + vector_dir_norm*np.array([[0.26,0.16,0.16,0.16]]).T
points_in_ew_frame_rotated = [np.dot(rot_mat_ew_to_lab,part[:,0:3].T).T for part in hull]


fig = go.Figure()





Plotter.scatter3d(fig,all_skin[::10,:],'skin', mode = 'markers',marker_dict =  {'color': 'black'}) 

Plotter.scatter3d(fig,points_in_ew_frame_rotated[0],'hull_body', mode = 'markers',marker_dict =  {'color': 'blue'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[1],'hull_rwing', mode = 'markers',marker_dict = { 'color': 'magenta'}) 
Plotter.scatter3d(fig,points_in_ew_frame_rotated[2],'hull_lwing', mode = 'markers',marker_dict = {'color':'cyan'}) 

marker_dict_point = {'size': 5, 'color': 'magenta'}

Plotter.scatter3d(fig,cm_point_lab[np.newaxis],'least square', mode = 'markers',marker_dict = {'size': 5, 'color': 'blue'}) 

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = {'size': 5, 'color': [[255,0,0],[0,255,0]]}) for origin,end,idx in zip(camera_center_lab, camera_pixel_lab,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = {'width': 3, 'color': 'red'}) for origin,end,idx in zip(camera_center_lab, camera_pixel_lab,range(4))]

fig.show()

